In [ ]:
#!/usr/bin/env bash

yugaware_url="ycs-gcp-platform-stable01"
auth_token="bcc36425-06b9-4bd2-b567-25370b9ec1d1"
customer_uuid="38b7df44-4993-4b8c-be89-b12b3c6f7921"

# Search the list of universes for any that have a instance tag with "yb_pause_at" and try to match it to the current UTC hour.
# if it matches, then ask it to pause...
# echo "Current hour is ${current_hour} UTC"

current_hour=`date +%H`

while IFS="," read -r uuid name
do
  echo "Universe: $name ($uuid) will be paused at ${current_hour} UTC..."
  curl --silent --insecure --request POST \
--url "https://${yugaware_url}/api/v1/customers/${customer_uuid}/universes/${uuid}/pause" \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${auth_token}" \
--data '{}'; echo
done < <(curl --silent --insecure --request GET \
--url https://${yugaware_url}/api/v1/customers/${customer_uuid}/universes \
--header 'Content-Type: text/json' \
--header "X-AUTH-YW-API-TOKEN: ${auth_token}" | jq -r '.[].universeDetails | select(.universePaused==false) | select(.clusters[].clusterType=="PRIMARY") | select(.clusters[].userIntent.instanceTags.yb_pause_at=='\""${current_hour}"\"') | [.universeUUID, .clusters[].userIntent.universeName] | join(",")')

In [ ]:
#!/usr/bin/env bash

yugaware_url="ycs-gcp-platform-stable01"
auth_token="bcc36425-06b9-4bd2-b567-25370b9ec1d1"
customer_uuid="38b7df44-4993-4b8c-be89-b12b3c6f7921"

# Search the list of universes for any that have a instance tag with "yb_resume_at" and try to match it to the current UTC hour.
# if it matches, then ask it to resume...
# echo "Current hour is ${current_hour} UTC"

current_hour=`date +%H`

while IFS="," read -r uuid name
do
  echo "Universe: $name ($uuid) will be resumed at ${current_hour} UTC..."
  curl --silent --insecure --request POST \
--url "https://${yugaware_url}/api/v1/customers/${customer_uuid}/universes/${uuid}/resume" \
--header 'Content-Type: application/json' \
--header "X-AUTH-YW-API-TOKEN: ${auth_token}" \
--data '{}'; echo
done < <(curl --silent --insecure --request GET \
--url https://${yugaware_url}/api/v1/customers/${customer_uuid}/universes \
--header 'Content-Type: text/json' \
--header "X-AUTH-YW-API-TOKEN: ${auth_token}" | jq -r '.[].universeDetails | select(.universePaused==false) | select(.clusters[].clusterType=="PRIMARY") | select(.clusters[].userIntent.instanceTags.yb_resume_at=='\""${current_hour}"\"') | [.universeUUID, .clusters[].userIntent.universeName] | join(",")')

In [ ]:
# every hour attempt to pause any universes with yb_pause_at set on that hour
0 * * * * /home/ssherwood/scripts/yb_pause_at.sh

# every hour on weekdays resume any universes with yb_resume_at set on that hour
0 * * * 1-5 /home/ssherwood/scripts/yb_resume_at.sh